# Data Cleaning

## Date: OCT 10, 2023

-- ------------------------


## Introduction

This notebook cleans the data for the lending club accepted loans, then exports the data as a parquet file. Due to the size of the dataset, the csv is read in chunks, with a random sample taken from each each chunk. Only fully paid and charged off / defaulted loans are sampled as current loans hold no value in classifying the target variable. This allows us to more efficiently load the data. Those samples are merged and will become the working dataset for the duration of the project. Unnecessary and leaky features are removed inplace to be more space efficient, formatted and null values removed. Finally the dataframe size is optimized to optimize space and computation efficiency, then exported. This file will be used for the EDA notebook

### Table-of-contents


1. [Introduction](#Introduction)
   - [Table-of-contents](#Table-of-contents)
   - [Import-Librarys](#Import-Librarys)
   - [Data Dictionary](#Data-Dictionary)
   - [Define-Functions](#Define-Functions)
   - [Load in the data](#Load-the-data)
3. [Data Cleaning](#Data-Cleaning)
   - [Initial Exploration](#Initial-Exploration)
   - [Feature Pruning](#Feature-Pruning)
   - [Explore Columns to drop](#Explore-Columns-to-drop)
   - [Dataframe Null Values](#Dataframe-Null-Values)
4. [Dataframe optimization](#Dataframe-optimization)
5. [Exploratory-Data-Analysis](Exploratory-Data-Analysis)
6. [Feature Engineering](#Feature-Engineering)
7. [Conclusion](#Conclusion)


### Import-Librarys

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import pdcast as pdc

from pathlib import Path

### Data-Dictionary

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [ ]:
#pathlib is used to ensure compatibility across operating systems
try:
    data_destination = Path('../Data/Lending_club/Lending Club Data Dictionary Approved.csv')
    dict_df = pd.read_csv(data_destination, encoding='ISO-8859-1')
    display(dict_df.iloc[:,0:2])
except FileNotFoundError as e:
    print(e.args[1])
    print('Check file location')

#### Define-Functions

In [ ]:
def map_emp_length(employment_length:str):
    '''
    Takes in employment length and returns an int for mapping

    :param employment_length: The employment length to be mapped
    :type employment_length: str

    :return: The int employment length should be mapped to
    :type return: int
    '''
    if employment_length == '< 1 year':
        return 0.5
    elif employment_length == '10+ years':
        return 10
    elif 'years' or 'year' in employment_length:
        return int(employment_length.split()[0])
    elif employment_length == '0':
        return 0
    else:
        return employment_length

When initially loading in the dataset, Pandas raised a DtypeWarning over mixed datatypes within various columns. Setting low_memory = False while breaking the CSV into chunks allows Pandas to load an entire chunk before guessing the data types. When the script to scrape the data dictionary is finished, the data dict can then be passed in instead of relying on pandas. The mixed_data_types function is stilled called as a sanity check.

In [ ]:
def mixed_data_types(df:pd.DataFrame) -> bool:
    '''
    Takes in a dataframe and checks for columns with mixed data types
    If none are found return False, else True
    
    :param df: The dataframe to be checked
    :type df: obj
    :return bool: True if found, false if none were found
    :type return: bool
    '''
    
    #loop through each column
    for column in df:

        #filter out int datatype coming from Nan and get unique data types
        unique_types = df[column].dropna(inplace=False).apply(type).unique()

        #if there are more than 1 datatype in a column
        if unique_types.size > 1:
            return True
    
    return False

#### Load the data

Due to the size of the dataset, it is read in chunks. After each chunk is read and checked for mixed data types, it is randomly sampled and then placed within a list. Only fully paid and /defaulted and charged off loans are taken, as current loans including late or in grace period loans do not hold any value in target variable prediction. This is done when loading in the data otherwise it becomes too large for memory. It is also more efficient. The different samples are then combined into a single sample representative of the whole dataset. EDA will be performed on this single sample.

In [ ]:
chunk_size = 5*100000
sample_size =  100000
random_state = 11

assert sample_size < chunk_size, f"Cannot take a sample of {sample_size} rows out of {chunk_size} rows"

print(f'Chunk size: {chunk_size} rows')
print(f'Rows to be sampled: {sample_size} rows')


sampled_dataframes = []
try:
    
    #path to the data. Should be under Data/Lending_club
    data_destination = Path('../Data/Lending_club/accepted_2007_to_2018Q4.csv')

    #split the csv into chunks and iterate over each chunk
    #set low_memory to false to force pandas to load entire columns before guessing data type
    with pd.read_csv(data_destination, chunksize=chunk_size, low_memory = False) as reader:
        
        for count,chunk in enumerate(reader):
            if mixed_data_types(df=chunk) == True:
                raise Exception("Mixed data types found")

            #define a list that includes only finished loan statuses
            finished_loan_status = ['Fully Paid',
                                    'Charged Off',
                                    'Does not meet the credit policy. Status:Fully Paid',
                                    'Does not meet the credit policy. Status:Charged Off',
                                    'Default']
                        
            #filter the dataframe for loans that are finished or null
            filtered_chunk = chunk.loc[chunk['loan_status'].isin(finished_loan_status) | chunk['loan_status'].isnull()]
            
            #sample the filtered df and append to list
            sampled_df = filtered_chunk.sample(n=sample_size, random_state=random_state)
            sampled_dataframes.append(sampled_df)
            
            print(f"{count} sampled dataframe shape: {sampled_df.shape}")
        print('Finished')

except FileNotFoundError as e:
    print(e.args[1])
    print('Check file name and location')
    
except Exception as e:
    print(e.args[1])

There are no duplicate datatypes within any columns. The random samples can be combined into a single sample dataframe. This sample will be used as the working dataset.

In [ ]:
sample_accepted_df = pd.concat(sampled_dataframes, ignore_index=False)

&nbsp;

## Data Cleaning

### Initial Exploration

***Display the first 5 rows*** 

In [ ]:
sample_accepted_df.head(5)

***Dataframe shape***

In [ ]:
rows, columns = sample_accepted_df.shape
print(f'Dataframe rows: {rows}')
print(f'Dataframe columns: {columns}')

***Dataframe info***

In [ ]:
sample_accepted_df.info()

Of the 151 columns, 113 are float64 and 38 are objects. The dataframe takes up approximatly 580 MB.
Note:
- The numeric columns are all float64 and the object columns. These columns can be optimized later to save memory space and decrease computation time by changing the datatypes.
- There is no datetime column.

***Describe Dataframe***

In [ ]:
sample_accepted_df.describe()

Some key points:

- Loan Amount
  
    - Average Loan Amount is ~ 15,000 USD with a standard deviation of 9240 USD, having a max of 40,000 USD and minimum of 500 USD. This follows LendingClubs  policies for minimum and maximum loan amounts.

- Funded amount
    - Nearly identical to the loan amount

- Funded amount by investors
    - Very similar to the  funded amount

- Interest Rate
    - The interest rates are quite high. An average of 13%, with a minimum of 5.3% and a maximum of 31%.


   

***Null Values***

Some rows are fully NaN values, aside from the id. This will cause issues when we try to inspect each column later. So we will drop `id` and the NaN rows, along with any other irrelevant columns including:  
- member_id
- url for the loan
- LC policy code
- title (information is already found under purpose)
- initial_list_status (what market it was listed under)

We will define a list to keep track of the columns we have dropped.

In [ ]:
dropped_columns = []

In [ ]:
drop_columns=['id', 'member_id', 'url', 'policy_code', 'title', 'initial_list_status']

# append the columns to drop
dropped_columns.extend(drop_columns)

sample_accepted_df.drop(columns=drop_columns, inplace=True)

In [ ]:
null_rows = sample_accepted_df.isnull().all(axis=1).sum()
print(f"Number of Null rows: {null_rows}")

In [ ]:
#drop rows that are all Nan
sample_accepted_df.dropna(how='all', inplace=True)

In [ ]:
null_rows = sample_accepted_df.isnull().all(axis=1).sum()
print(f"Number of Null rows: {null_rows}")

&nbsp;

---------------------------------------------

### Feature Pruning

We will exclude any leaky features, non relevant features and any features that were not present in the original loan application. This will be done by grouping the features together to better explain why they are being dropped. 

#### ***Irrelevant columns***

***Secondary Applicants Information***

The columns for the secondary applicants are largely nulls, so we will drop them.

In [ ]:
nulls_percent = (sample_accepted_df['sec_app_mort_acc'].isnull().sum()/sample_accepted_df.shape[0]*100)
print('Percentage of null rows for secondary applicants: ', nulls_percent.round(2), '%')

In [ ]:
sample_accepted_df['application_type'].value_counts()

In [ ]:
#get the index of the loans where application_type is a joint application
rows_to_remove = sample_accepted_df.loc[sample_accepted_df['application_type'] == 'Joint App'].index

# drop the loans
sample_accepted_df.drop(rows_to_remove, inplace=True)

# drop the related columns
drop_columns = ['revol_bal_joint', 'sec_app_fico_range_low', 
                'sec_app_fico_range_high', 'sec_app_earliest_cr_line',
                'sec_app_inq_last_6mths', 'sec_app_mort_acc',
                'sec_app_open_acc', 'sec_app_revol_util', 
                'sec_app_open_act_il', 'sec_app_num_rev_accts', 
                'sec_app_chargeoff_within_12_mths', 'sec_app_collections_12_mths_ex_med',
                'sec_app_mths_since_last_major_derog',
                'verification_status_joint', 'dti_joint',
                'annual_inc_joint']

# append the columns to drop
dropped_columns.extend(drop_columns)
sample_accepted_df.drop(columns=drop_columns, inplace=True)

We will still keep the flag of whether the application was a joint or individual application since there are no nulls and the information could be useful in our analysis. 

***Hardship Loans***

Hardship loans add 15 columns of complexity, are largely nulls and leak the loan outcome. We will drop these columns and loans if they exist in our dataset, and limit our analysis to non hardship loans.

In [ ]:
#fetch the value counts for the for the hardships flags
hardships = sample_accepted_df['hardship_flag'].value_counts()
display(hardships)

#if there are loans with the yes hardship flag
if 'Y' in hardships:
    #get the count of hardship loans
    yes_hardship_count = hardships.iloc[1]
    print(f'The hardship loans represent only {(yes_hardship_count/sample_accepted_df.shape[0])*100}% of the dataset')

    #get the index of the hardship loans
    rows_to_remove = sample_accepted_df.loc[sample_accepted_df['hardship_flag'] == 'Y'].index

    #drop the loans
    sample_accepted_df.drop(rows_to_remove, inplace=True)

    #check the rows have been dropped
    assert sample_accepted_df['hardship_flag'].value_counts().shape[0] == 1
    print('Hardship loans and associated columns have been dropped')

else:
    print('There are no hardship loans.')
    
drop_columns = ['hardship_flag', 'hardship_type',
                'hardship_reason', 'hardship_status',
                'hardship_amount', 'hardship_start_date',
                'hardship_end_date', 'deferral_term',
                'hardship_length', 'hardship_dpd',
                'hardship_loan_status', 'payment_plan_start_date',
                'orig_projected_additional_accrued_interest', 'hardship_payoff_balance_amount',
                'hardship_last_payment_amount']

# append the columns to drop
dropped_columns.extend(drop_columns)
sample_accepted_df.drop(columns = drop_columns, inplace=True)
print('Hardship columns have been dropped')

***Employee Title***

In [ ]:
unique_emp_titles = sample_accepted_df['emp_title'].nunique()
print(f'Number of unique employment titles: {unique_emp_titles}')

There are too many unique Employee titles to attempt any sort of grouping or encoding for now. In the future we could use NLP or an external API to group the Employee Title.

In [ ]:
# append the columns to drop
dropped_columns.append('emp_title')

sample_accepted_df.drop(columns = 'emp_title', inplace=True)

***Loan Status***

Any current loans have already been dropped when reading in the data. We can now finish grouping the completed loans.

More information on the loan status's can be found here:  
https://www.lendingclub.com/help/investing-faq/what-do-the-different-note-statuses-mean  
https://www.fintechnexus.com/policy-code-2-loans-lending-club/

In [ ]:
sample_accepted_df['loan_status'].value_counts()

The "Does not meet the credit policy" means when the loans were made under a different credit card policy, that does not meet the current policy. This has affect on the loans themselves, so they can be grouped with their counter parts. Charged off and Defaulted can also been grouped together.

In [ ]:
status_mapping = {
    "Fully Paid": "Fully Paid",
    "Does not meet the credit policy. Status:Fully Paid": "Fully Paid",
    "Does not meet the credit policy. Status:Charged Off": "Charged Off/Default",
    "Charged Off": "Charged Off/Default",
    "Default": "Charged Off/Default",
}

#map the loans
sample_accepted_df['loan_status'] = sample_accepted_df['loan_status'].map(status_mapping)

Check the mapping has worked:

In [ ]:
sample_accepted_df['loan_status'].value_counts()

The mapping was successful, we are not left with only successful and failed loans.

***State / Zip Code***

We have 2 geographical features. We will drop both of them for now as they will add too much complexity to the model. However, in the future we can perhaps use a 3rd party api and introduce mean or median income data by region, allowing us to capture some of that geographical data.

In [ ]:
display(sample_accepted_df['addr_state'].value_counts())
print('-'*20)
display(sample_accepted_df['zip_code'].value_counts())

In [ ]:
#drop_columns = ['addr_state', 'zip_code']

# append the columns to drop
#dropped_columns.extend(drop_columns)
#sample_accepted_df.drop(columns = drop_columns, inplace=True)

***Description***

In [ ]:
unique_desc_titles = sample_accepted_df['desc'].nunique()
print(f'Number of unique descriptions: {unique_desc_titles}')

There are too many unique descriptions to create dummy variables. We can drop this column

In [ ]:
drop_columns = ['desc']

# append the columns to drop
dropped_columns.extend(drop_columns)
sample_accepted_df.drop(columns = drop_columns, inplace=True)

#### Leaky columns

We will remove any columns that can leak the outcome of the application ie, any data the originates after a loan has been funded or rejected.  
The columns we have dropped so far:

In [ ]:
print('Columns dropped so far: ')
print(dropped_columns)

***Loan Grade***

Loan grade is calculated after the loan is given, so we can drop both `grade` and `sub_grade`.

In [ ]:
drop_columns = ['grade','sub_grade']

# append the columns to drop
dropped_columns.extend(drop_columns)
sample_accepted_df.drop(columns=drop_columns, inplace=True)

***Other features to drop***

We can remove any columns that:  
- describe payments made toward the loan

In [ ]:
drop_columns =  ['total_pymnt', 'total_rec_prncp',
                 'total_rec_int', 'total_rec_late_fee',
                 'last_pymnt_d', 'last_pymnt_amnt', 
                 'next_pymnt_d', 'total_pymnt_inv']

# append the columns to drop
dropped_columns.extend(drop_columns)
sample_accepted_df.drop(columns =drop_columns, inplace=True)

- describe debt collection or recovery

In [ ]:
drop_columns = ['collection_recovery_fee', 
                'collection_recovery_fee', 'recoveries']

dropped_columns.extend(drop_columns)
sample_accepted_df.drop(columns =drop_columns, inplace=True)

- loan attributes post acceptance

In [ ]:
drop_columns=['out_prncp', 'out_prncp_inv',
              'pymnt_plan', 'disbursement_method',
              'last_credit_pull_d',
              'debt_settlement_flag_date', 'settlement_term',
              'num_tl_120dpd_2m', 'num_tl_30dpd']

dropped_columns.extend(drop_columns)
sample_accepted_df.drop(columns=drop_columns, inplace=True)

- any settlement information

In [ ]:
drop_columns=['debt_settlement_flag', 'settlement_status',
              'settlement_date', 'settlement_amount',
              'settlement_percentage']

dropped_columns.extend(drop_columns)
sample_accepted_df.drop(columns=drop_columns, inplace=True)

- other columns

In [ ]:
drop_columns = ['max_bal_bc', 'open_rv_24m',
                'open_rv_12m', 'inq_fi',
                'total_bal_il', 'inq_last_12m',
                'open_il_24m', 'open_il_12m',
                'open_act_il', 'total_cu_tl',
                'open_acc_6m', 'il_util','mths_since_rcnt_il',
                'all_util']

dropped_columns.extend(drop_columns)
sample_accepted_df.drop(columns=drop_columns, inplace=True)

### Feature engineering

***Term***

Convert from str to int

In [ ]:
sample_accepted_df['term'].value_counts()

In [ ]:
#remove the "months" text and convert to int
sample_accepted_df['term'] = sample_accepted_df['term'].str.extract('(\d+)').astype('int32')

In [ ]:
sample_accepted_df['term'].value_counts()

***Emp_Length***

As employment length is ordinal, we will map greater than 10 years to 10, less than 1 year to 0.5 as to differentiate it between 1 and 0, preserving that information.

In [ ]:
sample_accepted_df['emp_length'].value_counts()

We will assume NA's as no employment

In [ ]:
sample_accepted_df['emp_length'].fillna(value='0',inplace=True)

Apply the mapping

In [ ]:
sample_accepted_df['emp_length'] = sample_accepted_df['emp_length'].apply(map_emp_length)

Check employment length has been updated

In [ ]:
sample_accepted_df['emp_length'].value_counts()

### Dataframe-Null-Values

------------------------------------------

In [ ]:
pd.set_option('display.max_rows', None)

We can calculate the percentages of null values by column

In [ ]:
(sample_accepted_df.isnull().sum()/sample_accepted_df.shape[0]*100).sort_values(ascending=False)

Note how there seems to be groupings of nulls. We will explore these groupings

&nbsp;

***Explore the groupings of nulls***

We will drop the loans for columns with less than 3% nulls. With a dataset this size, a few loans won't affect our analysis. Furthermore, majority of the features within these rows are nulls. There is a noticable gap at 3% which is why we will we choose it for our cutoff.

In [ ]:
cutoff = 3 #percent

#get the percentages of nulls for each column 
null_percentages = (sample_accepted_df.isnull().sum() / sample_accepted_df.shape[0]) * 100

#get the filtered columns
filtered_columns = null_percentages[null_percentages < cutoff].index.tolist()

#drop the loans with nulls for the filtered columns
sample_accepted_df_cleaned = sample_accepted_df.dropna(subset=filtered_columns, inplace=True)

We will also drop any columns that are majority nulls.

In [ ]:
column_cutoff = 10 #percent

#get the percentages of nulls for each column 
null_percentages = (sample_accepted_df.isnull().sum() / sample_accepted_df.shape[0]) * 100

#get the filtered columns
filtered_columns = null_percentages[null_percentages > column_cutoff].index.tolist()
dropped_columns.extend(filtered_columns)

#drop the filtered columns
sample_accepted_df.drop(columns=filtered_columns, inplace=True)

This leaves us with the the following column nulls:

In [ ]:
(sample_accepted_df.isnull().sum()/sample_accepted_df.shape[0]*100).sort_values(ascending=False)

We can now work through each grouping, starting with the samllest.

***acc_open_past_24mths***

In [ ]:
null_rows = sample_accepted_df[sample_accepted_df['acc_open_past_24mths'].isnull()]
null_rows.head()

In [ ]:
null_rows['issue_d'].value_counts()

Notice the date for the loans. The loans made early in lendingclubs history make up the majority of nulls for the remaining columns. This is impart due to lendingclub frequently updating their api, adding new fields, while the loans that are already recorded are filled with NaN values. Since our analysis is based on the exact combination of features for a loan, it simply does not make sense to keep these loans as there is no accurate way to impute the many missing values. We can remove the associated rows or features. Although this may add some recency bias, as we are narrowing our analysis to more recent loans that may not have as varied economic conditions among other factors, we will drop the rows due to our dataset size.  
Example:  
https://www.fintechnexus.com/lending-club-adds-15-new-fields-and-folio-introduces-a-true-secondary-market-api/

In [ ]:
sample_accepted_df.dropna(subset=['annual_inc', 'total_acc', 
                                  'tax_liens', 'chargeoff_within_12_mths', 
                                  'pub_rec_bankruptcies', 'total_bal_ex_mort',
                                  'tot_hi_cred_lim', 'avg_cur_bal', 
                                  'pct_tl_nvr_dlq', 'mo_sin_old_il_acct', 'bc_util', 'percent_bc_gt_75'
                                 ], inplace=True)

In [ ]:
(sample_accepted_df.isnull().sum()/sample_accepted_df.shape[0]*100).sort_values(ascending=False)

We have no more null values

### Dataframe Optimization

The library used to optimize the dataframe shape minimize to int8, which is not supported by parquet files. The code is left for reusability in the case someone wants to export as csv.

In [ ]:
#sample_accepted_df = pdc.downcast(sample_accepted_df)
#print(sample_accepted_df.info())
# Infer minimum schema for DataFrame.
#schema = pdc.infer_schema(sample_accepted_df)
#print(schema)
#sample_accepted_df.shape

Instead we will simply downcast the datatypes to int and float32 as this is more than enough precision for our data. 

In [ ]:
#downcast all float columns to float32
float_cols = sample_accepted_df.select_dtypes(include='float64').columns
for col in float_cols:
    sample_accepted_df[col] = pd.to_numeric(sample_accepted_df[col], downcast='float')

#downcast all int columns to int32
int_cols = sample_accepted_df.select_dtypes(include='int64').columns
for col in int_cols:
    sample_accepted_df[col] = sample_accepted_df[col].astype('int32')


In [ ]:
sample_accepted_df.dtypes

### Export Dataframe

***Export the dataframe for EDA***

In [ ]:
export_destination = Path('../Data/Lending_club/eda_cleaned')
sample_accepted_df.to_parquet(export_destination)
print('Cleaned data to be used for EDA has been exported')

***Export the dataframe for Models***

Drop any Leaky columns left over from EDA

In [ ]:
#drop leaky columns / unwanted columns
drop_columns=['funded_amnt', 'funded_amnt_inv', 'fico_range_low', 'fico_range_high', 'last_fico_range_high', 'last_fico_range_low']
dropped_columns.extend(drop_columns)
sample_accepted_df.drop(columns = drop_columns, inplace=True)

# drop categorical columns with too many categories for one hot encoding
drop_columns=['issue_d', 'earliest_cr_line', 'zip_code', 'addr_state']
dropped_columns.extend(drop_columns)
sample_accepted_df.drop(columns = drop_columns, inplace=True)

print('The final list of columns dropped : ')
print(dropped_columns)

Map **Successful loans to 1**, and **Defaulted or Charged Off loans to 0** in our target column.

In [ ]:
sample_accepted_df['loan_status'] = sample_accepted_df['loan_status'].apply(lambda x: 1 if x == 'Fully Paid' else 0)

In [ ]:
export_destination = Path('../Data/Lending_club/model_cleaned')
sample_accepted_df.to_parquet(export_destination)
print('Cleaned data to be used for modelling has been exported')

### Conclusion

In this notebook, we have completed a rudimentary cleaning of the lendingclub dataset. A random sample of 500,000 rows was taken from the dataset and cleaned. We have have dealt with any missing information stemming from changes in lendingclub's api over the years, and any other NaN values. Any features that could leak the outcome of the loan, were irrelevant, or added unnecessary complexity were have also been dropped. Some rudimentary feature engineering has been conducted but this will be expanded on later. Finally, the cleaned dataset is written to a parquet file.   

Note:   
- Although the leaky features were carefully reviewed, we will check the feature weights when performing our baseline logitist regression model to confirm that some leaky features haven't been kept. 
- We have removed the earlier loans completed in lendingclubs history due to api changes and large amount of null values. This restricts our dataset to a more recent timeframe, which could introduce a recency bias, given how sensitive loans are on economic conditions over this shorter period, as will be shown in EDA.
- Some features were kept for the sake of EDA, but will be dropped later. 

### Resources used:

- https://stackoverflow.com/questions/51325601/how-to-stop-my-pandas-data-table-from-being-truncated-when-printed